In [1]:
import os
from langchain.document_loaders import PyMuPDFLoader
# from langchain.vectorstores import SimpleVectorStore
from langchain.embeddings import OpenAIEmbeddings

In [2]:
def load_pdfs_from_folder(folder_path):
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
    documents = []
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(folder_path, pdf_file)
        loader = PyMuPDFLoader(pdf_path)
        docs = loader.load()
        documents.extend(docs)
    
    return documents

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
folder_path = r"vibration"
docs = load_pdfs_from_folder(folder_path)
chunked_docs = splitter.split_documents(docs)

In [4]:
len(chunked_docs)

130

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
 

db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))

e:\Learning\RAG\venv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
e:\Learning\RAG\venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
import faiss

# Retrieve the underlying FAISS index
faiss_index = db.index

# Save the FAISS index to a file
faiss.write_index(faiss_index, "faiss_index.index")

print("FAISS index saved to faiss_index.index")

FAISS index saved to faiss_index.index


In [9]:
import pickle

with open("chunked_docs.pkl", "wb") as f:
    pickle.dump(chunked_docs, f)

In [10]:
import faiss
import pickle
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Load the FAISS index from the file
faiss_index = faiss.read_index("faiss_index.index")

# Assuming you need to reconstruct the FAISS vector store
# You need to load the chunked_docs separately
with open("chunked_docs.pkl", "rb") as f:
    chunked_docs = pickle.load(f)

# Create the FAISS vector store with the loaded index and documents
db_loaded = FAISS(index=faiss_index, embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))

print("FAISS index and documents loaded successfully.")


TypeError: FAISS.__init__() got an unexpected keyword argument 'embeddings'

In [27]:
def retrieve_from_vector_store(vector_store, query, k=5):
    # Perform the retrieval operation
    results = vector_store.similarity_search(query, k=k)
    return results

In [33]:

# Example query
query = "What is the spring?"
results = retriever
# print("Retriever created successfully.")


# for result in results:
print(results[0])
    

page_content='to the force acting along its length:
F = k(x −u)
(2.1)
The ideal spring is considered to have no mass;
thus, the force acting on one end is equal and
2.1
FIGURE 2.1
Linear spring.' metadata={'source': 'vibration\\Vibration Theory.pdf', 'file_path': 'vibration\\Vibration Theory.pdf', 'page': 0, 'total_pages': 32, 'format': 'PDF 1.4', 'title': '8434_Harris_02_b.qxd', 'author': 'Prepress 1', 'subject': '', 'keywords': '', 'creator': 'QuarkXPressª 4.11: PSPrinter 8.3.1', 'producer': 'Acrobat Distiller 4.05 for Macintosh', 'creationDate': 'D:20010920123819Z', 'modDate': "D:20050202111026-05'00'", 'trapped': ''}


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "HuggingFaceH4/zephyr-7b-beta"

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
# )

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 8/8 [01:18<00:00,  9.82s/it]


In [7]:
# Specify directories to save the model and tokenizer
model_save_path = "./saved_model"
tokenizer_save_path = "./saved_tokenizer"

# Save the model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model saved to {model_save_path}")
print(f"Tokenizer saved to {tokenizer_save_path}")

Model saved to ./saved_model
Tokenizer saved to ./saved_tokenizer


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "HuggingFaceH4/zephyr-7b-beta"

# Paths where the model and tokenizer were saved
model_save_path = "./saved_model"
tokenizer_save_path = "./saved_tokenizer"

# Load the model and tokenizer from the saved paths
modelsave = AutoModelForCausalLM.from_pretrained(model_save_path)
tokenizersave = AutoTokenizer.from_pretrained(tokenizer_save_path)

Loading checkpoint shards:  33%|███▎      | 2/6 [00:02<00:05,  1.43s/it]

: 

In [1]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

e:\Learning\RAG\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'model' is not defined